# Linear pendulum simulation

## The ODE

When a pendulum has small-amplitude swings, the ODE for its motion is
$\theta''(t) = -\dfrac{g}{L}\theta(t)$,

Since it's a 2nd order ODE, we'll transform this into a system of two first order ODEs as follows

$$y_1 = \theta(t)$$

$$y_2 = \theta' = \omega(t)$$

$$\rightarrow \mathbf{y} = \left[\array{\theta \\ \theta'}\right]$$

Then

$$\mathbf{y}' = \left[\array{y_2 \\ -\dfrac{g}{L}y_1}\right] = \left[\array{0 & 1 \\ -\dfrac{g}{L} & 0}\right]\left[\array{y_1 \\ y_2}\right]$$

The eigenvalues for this system correspond to the natural frequency of the pendulum, $\lambda = \pm i\sqrt{\frac{g}{L}}$, or a natural frequency of $\omega = \sqrt{\frac{g}{L}}$.

### Numerical solution

The code below solves this problem using RK4, AB2, Leapfrog and BDF2 schemes

The multistep schemes require a startup time step. Since RK4 is self-starting and is computed here anyways, each scheme uses the RK4 result to advance the first step before switching to its own multistep update rule.

Try running the code for different time steps and think about the following questions:
1. Which schemes are stable? Conditionally stable? Unstable?
2. What differences do you see in amplitude (peak height) and phase shift (peak position) for each scheme?
3. Why is not not advisable to use forward Euler for the start-up scheme for the multistep methods on this linear pendulum problem?

## Code implementation

In [ ]:
# Import libraries
import numpy as np
import matplotlib.pyplot as plt

# Constants
g = 9.81
L = 1
w = np.sqrt(g/L) # Frequency, eigenvalue imag coef

# f(y) for the pendulum system
A = np.array([[0, 1],[-g/L, 0]])
f = lambda y: A@y

# Initial conditions and time step setup
y0 = np.array([0.1, 0])
h = .05
T = 50
N = round(T/h)
t = np.linspace(0, T, N+1)

# Set up solution vectors for each of the methods
yrk4  = np.zeros((2,N+1))
ylf   = np.zeros((2,N+1))
yab2  = np.zeros((2,N+1))
ybdf2 = np.zeros((2,N+1))

# Set initial conditions
yrk4[:,0]  = y0
ylf[:,0]   = y0
yab2[:,0]  = y0
ybdf2[:,0] = y0

# Time step loop - takes N steps
for n in range(N):
    
    # RK4 update
    k1 = f(yrk4[:,n])
    k2 = f(yrk4[:,n] + 0.5*h*k1)
    k3 = f(yrk4[:,n] + 0.5*h*k2)
    k4 = f(yrk4[:,n] + h*k3)
    yrk4[:,n+1] = yrk4[:,n] + h*(k1/6 + k2/3 + k3/3 + k4/6)
    
    # Multistep schemes: Each requires a different method to start up
    if(n==0):

        # Use the RK4 result since we have it already
        ylf[:,n+1]   = yrk4[:,n+1]
        yab2[:,n+1]  = yrk4[:,n+1]
        ybdf2[:,n+1] = yrk4[:,n+1]

    # Use the appropriate scheme for the 2nd step onwards
    else:
        # Leapfrog update
        ylf[:,n+1] = ylf[:,n-1] + 2*h*f(ylf[:,n])
        # AB2 update
        yab2[:,n+1] = yab2[:,n] + 1.5*h*f(yab2[:,n]) - 0.5*h*f(yab2[:,n-1])
        # BDF2 update
        ybdf2[:,n+1] = np.linalg.solve(np.eye(2) - (2/3)*h*A, (4/3)*ybdf2[:,n] - (1/3)*ybdf2[:,n-1])


# Check stability for the explicit schemes
print('h = ', h)
print('hmax RK4      = ', 2.83/w)
print('hmax Leapfrog = ', 1/w)
print('hmax AB2      = ', 0)


### Plotting

First let's examine the solution over the first 10 seconds, $t\in[0, 10]$.

In [ ]:
# True solution
theta_true = y0[0]*np.cos(t*np.sqrt(g/L)) 
omega_true = -np.sqrt(g/L)*y0[0]*np.sin(t*np.sqrt(g/L))

# Indices corresponding to times t <= 10
I = (t<=10)

# Plot
plt.figure()
plt.plot(t[I],yrk4[0,I],label='RK4')
plt.plot(t[I],ylf[0,I],label='Leapfrog')
plt.plot(t[I],yab2[0,I],label='AB2')
plt.plot(t[I],ybdf2[0,I],label='BDF2')
plt.plot(t[I],theta_true[I], '--', label='True soln')
plt.grid()
plt.xlabel('t')
plt.ylabel(r'$\theta(t)$')
plt.title(r'$\theta(t)$ for h = '+str(h))
plt.legend()

Now let's look at the solutions for $t\in[40, 50]$:

In [ ]:
# Indices corresponding to times t >= 40
I = (t>=40)

# Plot
plt.figure()
plt.plot(t[I],yrk4[0,I],label='RK4')
plt.plot(t[I],ylf[0,I],label='Leapfrog')
plt.plot(t[I],yab2[0,I],label='AB2')
plt.plot(t[I],ybdf2[0,I],label='BDF2')
plt.plot(t[I],theta_true[I], '--', label='True soln')
plt.grid()
plt.xlabel('t')
plt.ylabel(r'$\theta(t)$')
plt.title(r'$\theta(t)$ for h = '+str(h))
plt.legend()